In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from astropy.visualization import ZScaleInterval, ImageNormalize, LinearStretch
import glob, os
from photometryExercise import standardPlot
from photutils.aperture import aperture_photometry, CircularAperture, CircularAnnulus
from photometryExercise import standardPlot
from astropy.stats import sigma_clipped_stats
from photutils.detection import DAOStarFinder
from photutils.aperture import CircularAperture, ApertureStats
from matplotlib.colors import LogNorm

In [ ]:
from bokeh.io import show, output_notebook
# set local host for bokeh (change the number to the one in the url at the top of the page)
localhost = 'localhost:8888'

In [ ]:
output_notebook()

In [ ]:
pip install astropy

In [ ]:
def show_fits(data, title=""):
    """
    stolen from the obs astro code but like the backend code they wrote which we didn't see
    """
    plt.figure(figsize=(8, 8))
    
    # Use ZScale for astronomical images
    interval = ZScaleInterval()
    vmin, vmax = interval.get_limits(data)
    norm = ImageNormalize(vmin=vmin, vmax=vmax, stretch=LinearStretch())
    
    plt.imshow(data, origin='lower', cmap='gray', norm=norm)
    plt.colorbar(label='Counts')
    plt.title(title)
    print(f"{title}: min={np.min(data):.1f}, max={np.max(data):.1f}, median={np.median(data):.1f}")
    print(f"Display range (ZScale): {vmin:.1f} to {vmax:.1f}")
    plt.show()
    
trim = 50  # idk what this should be lol

In [ ]:
# a function that takes the total flux as input and returns a magnitude value

def flux_to_mag(C):

    # check if the counts are negative
    if C <= 0:
        print("Counts are non-positive")
        return np.nan
        
    return - 2.5 * np.log10(C)

In [ ]:
# a function as well that estimates the error on the magnitude measuments

# C is the counts from the object
# s is the counts from the sky

def error_in_mag(C, s):

    # calculate the total counts within the aperture t
    t = C + s

    # calculate the final area of the objects counts by summing in quadrature of the errors
    del_C = np.sqrt(t + s)

    # calculate and return the magnitude error
    return - 2.5 / np.log(10) * (del_C / C)

In [ ]:
# write a function to obtain all the instrumental magnitudes and the errors 
def instrumental_magnitudes(f, xcor, ycor, rAperture=5):
    # where f is the individual file for each standard

    standard = fits.open(f)[0].data

    # create the ciruclar aperture and sky annulus
    rSkyInner = 1.5 * rAperture
    rSkyOuter = 2 * rAperture

    apertureStandard = CircularAperture((xcor, ycor), r=rAperture)
    annulusSky = CircularAnnulus((xcor, ycor), r_in=rSkyInner, r_out=rSkyOuter)

    # make an estimation of the sky values in the defined annulus
    # aperture_photometry returns an astropy.table with the center of the aperture and 
    # the total flux on the aperture ('aperture_sum')
    sky = aperture_photometry(standard, annulusSky)
    #print(sky)

    # to recover the mean sky value, weight the total flux by the area of the annulus
    area = np.pi * (rSkyOuter ** 2 - rSkyInner **2)
    skyFlux = sky['aperture_sum'].value[0]
    meanSkyValue = skyFlux / area
    #print(f'The sky flux is {skyFlux}')
    #print(f'The mean sky value is {meanSkyValue}')

    # remove the sky value
    standard = aperture_photometry(standard - meanSkyValue, apertureStandard)

    # extract value from astropy.table
    standardFlux = standard['aperture_sum'].value[0]
    #print(f'The total flux on aperture is {standardFlux}')

    # calculating instrumental magnitude and its errors and append to list
    instrumental_magnitude = flux_to_mag(standardFlux)
    instrumental_magnitude_error = error_in_mag(standardFlux, skyFlux)

    return instrumental_magnitude, instrumental_magnitude_error

In [ ]:
# write a function to obtain the airmass
def airmasses(f):
    # where f is the individual file for each standard

    with fits.open(f) as hdul:
        header = hdul[0].header

    altitude = header['ALTITUDE']
    zenith_angle = 90.0 - altitude

    # convert to radians
    zenith_angle_rad = np.deg2rad(zenith_angle)

    # calcualte airmass
    airmass = 1.0 / np.cos(zenith_angle_rad)
    #print(airmass)

    return airmass

In [ ]:
# write a function to obtain the coordinates to follow a star from initial coordinates
def coordinates(f, xcor, ycor, aperture_radius = 5, sigma = 3.0, fwhm = 3.0):

    # open data
    hdul = fits.open(f)
    image_data = hdul[0].data

    # Detect stars in this image
    mean, median, std = sigma_clipped_stats(image_data, sigma=3.0)
    daofind = DAOStarFinder(fwhm=fwhm, threshold=sigma*std)
    sources = daofind(image_data - median)

    # Find the detected star closest to your expected position
    distances = np.sqrt((sources['xcentroid'] - xcor)**2 + (sources['ycentroid'] - ycor)**2)
    idx = np.argmin(distances)
    star_pos = (sources['xcentroid'][idx], sources['ycentroid'][idx])

    # Photometry at the detected position
    #aperture = CircularAperture(star_pos, r=aperture_radius)
    #aperstats = ApertureStats(image_data, aperture)
    hdul.close()

    return star_pos

In [ ]:
# a function to show the plot and location of the star

def star_location_plot(f, xcor, ycor, xmin=700, xmax=1200, ymin=700, ymax=1200):

    fwhm = 3.0
    sigma = 3.0

    # open data
    hdul = fits.open(f)
    image_data = hdul[0].data

    # Detect stars in this image
    mean, median, std = sigma_clipped_stats(image_data, sigma=3.0)
    daofind = DAOStarFinder(fwhm=fwhm, threshold=sigma*std)
    sources = daofind(image_data - median)

    # Find the detected star closest to your expected position
    distances = np.sqrt((sources['xcentroid'] - xcor)**2 + (sources['ycentroid'] - ycor)**2)
    idx = np.argmin(distances)
    star_pos = (sources['xcentroid'][idx], sources['ycentroid'][idx])

    x2 = 98
    positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
    apertures = CircularAperture(positions, r=6)
    plt.figure()
    plt.imshow(image_data, origin='lower', norm=LogNorm(vmin=np.median(image_data), vmax=np.percentile(image_data, x2)), interpolation='nearest')
    apertures.plot(color='blue', lw=1.0, alpha=0.5)

    target_aperture = CircularAperture(star_pos, r=6)
    target_aperture.plot(color='red', lw=1.0, alpha=0.5) # Use a slightly thicker line (lw=2.5) for visibility
    
    plt.xlim(700, 1200)  # x-axis pixel range
    plt.ylim(700, 1200)  # y-axis pixel range
    
    print(f, star_pos[0], star_pos[1])
    
    plt.show()

In [ ]:
# path settings
BASE_DIR = "data"

# standard stars + target fits files
STANDARD_DIR = os.path.join(BASE_DIR, "standards")

# individual standard stars + target
STAR1 = os.path.join(STANDARD_DIR, "star1")
STAR2 = os.path.join(STANDARD_DIR, "star2")
STAR3 = os.path.join(STANDARD_DIR, "star3")
STAR4 = os.path.join(STANDARD_DIR, "star4")
TARGET = os.path.join(STANDARD_DIR, "target")

# retrieving the data from the folders
star1_files = sorted(glob.glob(os.path.join(STAR1, "*.fits")))
star2_files = sorted(glob.glob(os.path.join(STAR2, "*.fits")))
star3_files = sorted(glob.glob(os.path.join(STAR3, "*.fits")))
star4_files = sorted(glob.glob(os.path.join(STAR4, "*.fits")))
target_files = sorted(glob.glob(os.path.join(TARGET, "*.fits")))

In [ ]:
# find coordinates for star 1
with fits.open(star1_files[3]) as hdul:
    #print(star1_files[0])
    image_data = hdul[0].data
    #show_fits(image_data)
    #standardPlot(image_data, notebook_url=localhost)

In [ ]:
# print out the locations of star1 images

# for the B filter
#for f in star1_files[:3]:
#    star_location_plot(f, 970.50, 982.54)

# for the other filters
#for f in star1_files[3:]:
#    star_location_plot(f, 986, 912)

In [ ]:
# write a loop to obtain the instrumental magnitude and its errors
# for star1

star1_ins_mag = []
star1_ins_mag_err = []
star1_airmass = []
star1_tidied_f1 = []

for f in star1_files[:3]:

    # found only using the first image: PIRATE_OSL_ROE_EXO1_star1_filter_B_2.fits
    xcor = 970.50
    ycor = 982.54

    # find the coordinates of the star using initial coordinates
    x, y = coordinates(f, xcor, ycor)

    # calculate the instrumental magnitudes 
    ins_mag, ins_mag_err = instrumental_magnitudes(f, x, y)

    # calculate the airmass
    airmass = airmasses(f)

    # print out values 
    prefix_to_remove = "data/standards/star1/"
    tidied_f1 = f.replace(prefix_to_remove, "")
    star1_tidied_f1.append(tidied_f1)
    print(tidied_f1, ins_mag, ins_mag_err, airmass)

    star1_ins_mag.append(ins_mag)
    star1_ins_mag_err.append(ins_mag_err)
    star1_airmass.append(airmass)

for f in star1_files[3:]:

    # found only using the first image: PIRATE_OSL_ROE_EXO1_star1_filter_B_2.fits
    xcor = 986.50
    ycor = 912.54

    # find the coordinates of the star using initial coordinates
    x, y = coordinates(f, xcor, ycor)

    # calculate the instrumental magnitudes 
    ins_mag, ins_mag_err = instrumental_magnitudes(f, x, y)

    # calculate the airmass
    airmass = airmasses(f)

    # print out values 
    prefix_to_remove = "data/standards/star1/"
    tidied_f1 = f.replace(prefix_to_remove, "")
    star1_tidied_f1.append(tidied_f1)
    print(tidied_f1, ins_mag, ins_mag_err, airmass)

    star1_ins_mag.append(ins_mag)
    star1_ins_mag_err.append(ins_mag_err)
    star1_airmass.append(airmass)

In [ ]:
# find coordinates for star 2
with fits.open(star2_files[0]) as hdul:
    print(star2_files[0])
    image_data = hdul[0].data
    #show_fits(image_data)
    #standardPlot(image_data, notebook_url=localhost)

In [ ]:
# print out the locations of star2 images

# for all filters
#for f in star2_files:
#    star_location_plot(f, 993.53, 979.93)

In [ ]:
# write a loop to obtain the instrumental magnitude and its errors
# for star2

star2_ins_mag = []
star2_ins_mag_err = []
star2_airmass = []
star2_tidied_f2 = []

# found only using the first image: PIRATE_OSL_ROE_EXO1_star2_filter_R_1.fits
xcor = 993.53
ycor = 979.93

for f in star2_files:

    # find the coordinates of the star using initial coordinates
    x, y = coordinates(f, xcor, ycor)

    # calculate the instrumental magnitudes 
    ins_mag, ins_mag_err = instrumental_magnitudes(f, x, y)

    # calculate the airmass
    airmass = airmasses(f)

    # print out values 
    prefix_to_remove = "data/standards/star2"
    tidied_f2 = f.replace(prefix_to_remove, "")
    star2_tidied_f2.append(tidied_f2)
    print(tidied_f2, ins_mag, ins_mag_err, airmass)

    star2_ins_mag.append(ins_mag)
    star2_ins_mag_err.append(ins_mag_err)
    star2_airmass.append(airmass)

In [ ]:
# find coordinates for star 3
with fits.open(star3_files[0]) as hdul:
    print(star3_files[0])
    image_data = hdul[0].data
    #show_fits(image_data)
    #standardPlot(image_data, notebook_url=localhost)

In [ ]:
# print out the locations of star3 images

# for all filters
for f in star3_files:
    star_location_plot(f, 995.10, 935.05)

In [ ]:
# write a loop to obtain the instrumental magnitude and its errors
# for star3

star3_ins_mag = []
star3_ins_mag_err = []
star3_airmass = []
star3_tidied_f3 = []

# found only using the first image: PIRATE_OSL_ROE_EXO1_star3_filter_I_2.fits
xcor = 995.10
ycor = 935.05

for f in star3_files:

    # find the coordinates of the star using initial coordinates
    x, y = coordinates(f, xcor, ycor)

    # calculate the instrumental magnitudes 
    ins_mag, ins_mag_err = instrumental_magnitudes(f, x, y)

    # calculate the airmass
    airmass = airmasses(f)

    # print out values 
    prefix_to_remove = "data/standards/star3"
    tidied_f3 = f.replace(prefix_to_remove, "")
    star3_tidied_f3.append(tidied_f3)
    print(tidied_f3, ins_mag, ins_mag_err, airmass)

    star3_ins_mag.append(ins_mag)
    star3_ins_mag_err.append(ins_mag_err)
    star3_airmass.append(airmass)

In [ ]:
# find coordinates for star 4
with fits.open(star4_files[0]) as hdul:
    print(star4_files[0])
    image_data = hdul[0].data
    #show_fits(image_data)
    #standardPlot(image_data, notebook_url=localhost)

In [ ]:
# print out the locations of star4 images

# for all filters
#for f in star4_files:
#    star_location_plot(f, 1059.54, 937.49)

In [ ]:
# write a loop to obtain the instrumental magnitude and its errors
# for star4

star4_ins_mag = []
star4_ins_mag_err = []
star4_airmass = []
star4_tidied_f4 = []

# found only using the first image: PIRATE_OSL_ROE_EXO1_star4_filter_B_1.fits
xcor = 1059.54
ycor = 937.49

for f in star4_files:

    # find the coordinates of the star using initial coordinates
    x, y = coordinates(f, xcor, ycor)

    # calculate the instrumental magnitudes 
    ins_mag, ins_mag_err = instrumental_magnitudes(f, x, y)

    # calculate the airmass
    airmass = airmasses(f)

    # print out values 
    prefix_to_remove = "data/standards/star4"
    tidied_f4 = f.replace(prefix_to_remove, "")
    star4_tidied_f4.append(tidied_f4)
    print(tidied_f4, ins_mag, ins_mag_err, airmass)

    star4_ins_mag.append(ins_mag)
    star4_ins_mag_err.append(ins_mag_err)
    star4_airmass.append(airmass)

In [ ]:
# find coordinates for wasp135b
with fits.open(target_files[0]) as hdul:
    print(target_files[0])
    image_data = hdul[0].data
    #show_fits(image_data)
    #standardPlot(image_data, notebook_url=localhost)

In [ ]:
# print out the locations of wasp135b images

# for all filters
#for f in target_files:
#    star_location_plot(f, 972.15, 954.72)

In [ ]:
# write a loop to obtain the instrumental magnitude and its errors
# for wasp135b

target_ins_mag = []
target_ins_mag_err = []
target_airmass = []
target_tidied_ft = []

# found only using the first image: PIRATE_OSL_ROE_EXO1_WASP135_Filter_B_1.fits
xcor = 972.15
ycor = 954.72

for f in target_files:

    # find the coordinates of the star using initial coordinates
    x, y = coordinates(f, xcor, ycor)

    # calculate the instrumental magnitudes 
    ins_mag, ins_mag_err = instrumental_magnitudes(f, x, y)

    # calculate the airmass
    airmass = airmasses(f)

    # print out values 
    prefix_to_remove = "data/standards/star4"
    tidied_ft = f.replace(prefix_to_remove, "")
    target_tidied_ft.append(tidied_ft)
    print(tidied_ft, ins_mag, ins_mag_err, airmass)

    target_ins_mag.append(ins_mag)
    target_ins_mag_err.append(ins_mag_err)
    target_airmass.append(airmass)

In [ ]:
star4_ins_mag = []
star4_ins_mag_err = []
star4_airmass = []
star4_tidied_f4 = []

In [ ]:
stars_ins_mag = [star1_ins_mag, star2_ins_mag, star3_ins_mag, star4_ins_mag]
stars_ins_mag_err = [star1_ins_mag_err, star2_ins_mag_err, star3_ins_mag_err, star4_ins_mag_err]
stars_airmass = [star1_airmass, star1_airmass, star1_airmass, star1_airmass]

In [ ]:
# write a function to calculate the mean instrumental magnitude of a filter for all stars vs airmass
def mean_ins_mag_vs_airmass(filter):

    # create empty lists to store data
    ins_mag = []
    ins_mag_err = []
    airmass = []   

    # Determine the index slice for the filter
    if filter == 'B':
        start_idx, end_idx = 0, 3
    elif filter == 'I':
        start_idx, end_idx = 3, 6
    elif filter == 'R':
        start_idx, end_idx = 6, 9
    elif filter == 'V':
        start_idx, end_idx = 9, 12
    else:
        # If the filter is not recognized, return empty lists
        print(f"Filter '{filter}' not recognized.")
        return [], []

    # iterate through all four standard stars
    for i in range(0,4):

        ins_m = stars_ins_mag[i]
        ins_m_err = stars_ins_mag_err[i]
        airm = stars_airmass[i]

        mean_ins_mag = np.mean(ins_m[start_idx:end_idx])
        mean_ins_mag_err = np.mean(ins_m_err[start_idx:end_idx]) # this is not how to do the errors
                                                                    # redo properly later
        mean_airmass = np.mean(airm[start_idx:end_idx])

        ins_mag.append(mean_ins_mag)
        ins_mag_err.append(mean_ins_mag_err)
        airmass.append(mean_airmass)
            
    return ins_mag, ins_mag_err, airmass

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def model(x, a, b):
    return a * x + b

In [ ]:
def perform_extinction_correction(magnitudes, airmasses, magnitude_errors, filter_name):
    # The extinction equation is: m_inst = m_0 + k * chi

    # make sure the errors are positive
    magnitude_errors = np.abs(magnitude_errors)

    # Fit the model to the data
    params, covariance = curve_fit(model, magnitudes, airmasses, sigma = magnitude_errors)
    
    # Extract the fitted parameters
    a_fit, b_fit = params
    cov_matrix = covariance
    
    # Calculate the parameter errors (standard deviations)
    a_error = np.sqrt(cov_matrix[0, 0])  # Uncertainty of a (slope)
    b_error = np.sqrt(cov_matrix[1, 1])  # Uncertainty of b (intercept)

    # Create the fitted curve
    x_fit = np.linspace(min(magnitudes), max(magnitudes), 100)
    y_fit = model(x_fit, *params)
    
    # Plot the data and the fitted line
    plt.scatter(magnitudes, airmasses)
    plt.plot(x_fit, y_fit, label="Best fit function", color='red')

    # plot the error bars
    #plt.errorbar(magnitudes, airmasses, yerr=magnitude_errors, fmt='o', label="Data", capsize=1)
    
    # Set labels and title
    plt.xlabel('Airmass ($\chi$)')
    plt.ylabel('Instrumental Magnitude ($m_{inst}$)')
    plt.title(f'Atmospheric Extinction Correction for {filter_name} Filter')
    plt.legend()
    
    # Show the plot
    plt.show()

    print("-" * 50)
    print(f"Filter: {filter_name}")
    print(f"Extinction Coefficient (k): {a_fit:.4f} ± {a_error:.4f}")
    #print(f"Outside-Atmosphere Magnitude (m0): {m0:.4f}")
    print("-" * 50)

    # where a_fit = k, a_error = k_error
    return a_fit, a_error

In [ ]:
# plot the instrumental magnitude vs airmass for B filter
ins_mag_B, ins_mag_err_B, airmass_B = mean_ins_mag_vs_airmass('B')
perform_extinction_correction(ins_mag_B, ins_mag_err_B, airmass_B, 'B')

In [ ]:
ins_mag_I, ins_mag_err_I, airmass_I = mean_ins_mag_vs_airmass('I')
perform_extinction_correction(ins_mag_I, ins_mag_err_I, airmass_I, 'I')

In [ ]:
ins_mag_R, ins_mag_err_R, airmass_R = mean_ins_mag_vs_airmass('R')
perform_extinction_correction(ins_mag_R, ins_mag_err_R, airmass_R, 'R')

In [ ]:
ins_mag_V, ins_mag_err_V, airmass_V = mean_ins_mag_vs_airmass('V')
perform_extinction_correction(ins_mag_V, ins_mag_err_V, airmass_V, 'V')

In [ ]:
# standard catalogue magntiudes for the four standard stars in each filter

# for standard star 1 (SA111-1965)
standard_1_B = 13.129 
standard_1_V = 11.419
standard_1_R = 10.468
standard_1_I = 9.589

# for standard star 2 (SA110 - 506)
standard_2_B = 11.880
standard_2_V = 11.312
standard_2_R = 10.977
standard_2_I = 10.660

# for standard star 3 (SA113_156)
standard_3_B = 11.750
standard_3_V = 11.224
standard_3_R = 10.921
standard_3_I = 10.606

# for standard star 4 (G26_7)
standard_4_B = 13.674
standard_4_V = 12.005
standard_4_R = 10.705
standard_4_I = 9.019

In [ ]:
# airmasses for the four standard stars in each filter

In [ ]:
# calculate the extinction coefficients

#for f in in instrumental_magnitude:
    